# Foundation Model Training API: Data validation and \$token estimation

This notebook illustrates the Databricks Foundation Model Training API. You can use this notebook to confirm that your data is in the correct format for training. This notebook can also help you estimate costs for the training process.

This notebook should not be used as part of the training process. Instead, you can run it before starting training to validate your data and help estimate the cost of the training tasks.

The inputs to this validation script are the same as the arguments to the Foundation Model Training API, shown below. 
```
cfg = {
    model: str,
    train_data_path: str,
    save_folder: str,
    *,
    task_type: Optional[str] = "INSTRUCTION_FINETUNE",
    eval_data_path: Optional[str] = None,
    eval_prompts: Optional[List[str]] = None,
    custom_weights_path: Optional[str] = None,
    training_duration: Optional[str] = None,
    learning_rate: Optional[float] = None,
    context_length: Optional[int] = None,
    experiment_trackers: Optional[List[Dict]] = None,
    disable_credentials_check: Optional[bool] = None,
    timeout: Optional[float] = 10,
    future: Literal[False] = False,
}
``` 

## Installation

In [ ]:
%pip uninstall -y llm-foundry

In [ ]:
dbutils.library.restartPython()

In [ ]:
%pip install --upgrade --no-deps git+https://github.com/mosaicml/llm-foundry.git@byod/data_validation
%pip install 'mosaicml[libcloud,wandb,oci,gcs]>=0.20.1,<0.21'
%pip install 'mlflow>=2.10,<3'
%pip install 'transformers>=4.38.2,<4.39'
%pip install "mosaicml-streaming>=0.7.4,<0.8"
%pip install -U datasets
%pip install omegaconf
%pip install einops
%pip install sentencepiece

In [ ]:
dbutils.library.restartPython()

In [ ]:
import os
import re
import json
import tempfile
import random
import numpy as np
import pandas as pd 
from collections import defaultdict
from argparse import ArgumentParser, Namespace
from typing import cast 

import datasets 

from llmfoundry.utils import (create_om_cfg, token_counts_with_collate, token_counts, 
        check_HF_datasets, is_hf_dataset_path, is_uc_delta_table,
        pandas_processing_fn, integrity_check, convert_text_to_mds, parse_args, 
        _args_str, plot_hist, dataframe_to_mds)

from llmfoundry.data.finetuning.tasks import (_validate_chat_formatted_example,
                                              _validate_prompt_response_formatted_example,
                                              _get_example_type, ChatFormattedDict, PromptResponseDict )
import transformers
transformers.logging.set_verbosity_error()

## Instruction fine tuning

### Foundation Model Training API arguments configuration

This section of the notebook is dedicated to setting up the parameters for the validation notebook. These parameters should be identical to what you specify in Foundation Model Training API. 

**Foundation Model Training API Arguments (FMT_API_args):**

- model: The model to be used for training. For example, 'EleutherAI/gpt-neox-20b'
- train_data_path: The path to the training data. It can be a Hugging Face dataset, a path to a `jsonl` file, or a Delta table.
- task_type: The type of task, either 'INSTRUCTION_FINETUNE' or 'CONTINUED_PRETRAIN'.
- training_duration: The number of epochs to run for the training process.
- context_length: The context length of the model, set to 2048. This is the number of tokens that the model considers for each training example.

**Temporary Data Path Configuration:**

- temporary_jsonl_data_path: Defines a filesystem path where temporary data related to the training process is stored. You need to make sure the path is not shared by other users on the cluster, as it costs contention.
- Environment variables for Hugging Face caches (HF_DATASETS_CACHE) are set to '/tmp/', directing dataset caching to a temporary directory.

**[Models supported by the Foundation Model Training API](https://docs.mosaicml.com/projects/mcli/en/latest/finetuning/finetuning.html#supported-models):**. 

You need to specify context length based on the model mapping below.
```
ft_models = {
  'meta-llama/Llama-2-13b-hf': 4096,
  'meta-llama/Llama-2-7b-hf': 4096,
  'meta-llama/Llama-2-70b-hf': 4096,
  'codellama/CodeLlama-7b-hf': 16384,
  'codellama/CodeLlama-13b-hf': 16384,
  'codellama/CodeLlama-34b-hf': 16384,
  'mistralai/Mistral-7B-v0.1': 32768,
}
```

In [ ]:
# Make sure you have write access to the ``home`` directory
home = os.path.join('/local_disk0', 'ift')
os.makedirs(home, exist_ok=True)
os.chdir(home)

In [ ]:
FMT_API_args = Namespace(
    model= 'meta-llama/Llama-2-7b-chat-hf', # Other examples: 'EleutherAI/gpt-neox-20b',
    train_data_path= 'mosaicml/dolly_hhrlhf/train',  # Other examples: '/path/to/train.jsonl', 'catalog.schema.table', 'iamroot/chat_formatted_examples/train', 
    task_type='INSTRUCTION_FINETUNE',
    training_duration=3,
    context_length=2048,
)

temporary_jsonl_data_path = os.path.join(home, 'ft_data_11Jan24_3/train')
os.environ['HF_DATASETS_CACHE'] = os.path.join(home, 'hf_cache')
os.makedirs(temporary_jsonl_data_path, exist_ok=True)
os.makedirs(os.environ['HF_DATASETS_CACHE'], exist_ok=True)

### Data loading

The instruction fine-tuning data requires the following format:

```
prompt: xxx
response or completion: yyy
```

Based on FMT_API_args.train_data_path, we select an ingestion method from one of the three options below:

- Option 1. Data is a JSONL file which stores in an object store supported by Composer.
- Option 2. Data is a Hugging Face dataset ID. Note you need to provide a split as well. 
- Option 3. Data is a Delta table. 

In [ ]:
raw_dataset = None

if is_hf_dataset_path(FMT_API_args.train_data_path):
    check_HF_datasets(FMT_API_args.train_data_path)
    dataset_id, split = '/'.join(FMT_API_args.train_data_path.split('/')[:2]), FMT_API_args.train_data_path.split('/')[-1]    
    raw_dataset = datasets.load_dataset(dataset_id, split=split)       
else:
    if is_uc_delta_table(FMT_API_args.train_data_path):    
        df = spark.read.table(FMT_API_args.train_data_path).toPandas()
        df.to_json(os.path.join(temporary_jsonl_data_path, 'data.jsonl'), orient='records', lines=True)
        raw_dataset = datasets.Dataset.from_pandas(df) 
        FMT_API_args.train_data_path = temporary_jsonl_data_path
    else: 
        # train_data_path is a jsonl file (local/remote)
        from composer.utils import dist, get_file, parse_uri 
        data_path = FMT_API_args.train_data_path 
        backend, _, _ = parse_uri(data_path)
        if backend not in ['', None]: # It's a remote path, download before loading it
            with tempfile.TemporaryDirectory() as tmp_dir:
                destination = os.path.join(tmp_dir, 'data.jsonl')
                get_file(data_path, destination)
                df = pd.read_json(destination, orient='records', lines=True)    
        else: 
            df = pd.read_json(data_path, orient='records', lines=True)    

        raw_dataset = datasets.Dataset.from_pandas(df)
        FMT_API_args.train_data_path = os.path.dirname(data_path)

if raw_dataset is None: 
    raise RuntimeError("Can't find a proper ingestion method")

### Data quality checks on the dataset


This section of the notebook performs a series of checks on the initial dataset to ensure its quality and expected format. This process ensures that the dataset adheres to the expected structure and contains the necessary keys for further processing. The checks are outlined below.

1. The total number of examples in the dataset is printed.
2. The first example from the dataset is displayed. This provides a quick glimpse into the data structure and format.
3. Data format validation:
- The dataset is expected to consist of dictionary-like objects (key-value pairs). A check is performed to validate this structure.
Each example in the dataset is examined for its compliance with the expected format.
4. Key presence validation:
- Allowed prompt and response keys, chat roles are defined in [llmfoundry](https://github.com/mosaicml/llm-foundry/blob/main/llmfoundry/data/finetuning/tasks.py): _ALLOWED_RESPONSE_KEYS and _ALLOWED_PROMPT_KEYS and _ALLOWED_ROLES.
- For prompt response dataset, the script checks for the presence of at least one prompt key and one response key in each example.
  - Prompt Validation: Each example is checked for the presence of keys defined in _ALLOWED_PROMPT_KEYS. If no valid prompt key is found, it is counted as a format error. 
  - Response Validation: Similarly, each example is checked for the presence of keys defined in _ALLOWED_RESPONSE_KEYS. An absence of a valid response key is also counted as a format error.
- For chat formatted dataset, the script checks if the message content is formatted valid by calling [_validate_chat_formatted_example](https://github.com/mosaicml/llm-foundry/blob/cffd75e94e5c53b1b14c67cd17e0916fecfd0e16/llmfoundry/data/finetuning/tasks.py#L130) helper function.  

Error Reporting:

If any format errors are found during the checks, they are reported.
A summary of errors is printed, categorizing them into types like data_type (non-dictionary data), missing_prompt, and missing_response.
If no errors are found, a congratulatory message is displayed, indicating that all checks have passed successfully.

In [ ]:
# Initial dataset stats
print("Num examples:", len(raw_dataset))
print("First example:")
for ex in raw_dataset: 
    print(ex)
    print() 
    break 

format_errors = defaultdict(int)

for example in raw_dataset:
    try: 
        example_format = _get_example_type(ex)
    except ValueError:
        format_errors["unknown example type"] += 1 
        continue 

    if example_format == 'chat':
        try: 
            chat_example = cast(ChatFormattedDict, example)
            _validate_chat_formatted_example(chat_example)
        except Exception as e:             
            format_errors['chat_format_error'] += 1  
            print(e)
            break 

    elif example_format == 'prompt_response':
        try:
            prompt_response_example: PromptResponseDict = cast(
                PromptResponseDict, example)
            _validate_prompt_response_formatted_example(prompt_response_example)
        except Exception as e: 
            format_errors['prompt_response_format_error'] += 1  
            print(e)
            break 
        
if format_errors:
    print("Oops! Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("Congratulations! No errors found")    

### Token estimation

Tokenize the raw dataset, calculate some statistics of the tokens, and estimate the overall cost based on default trainining duration
We will iterate over the dataloader and sum the number of tokens from each batch. 

In [ ]:
n_epochs = FMT_API_args.training_duration if FMT_API_args.training_duration is not None else 1 
batch_tokens = token_counts_with_collate(FMMT_API_args)
n_billing_tokens_in_dataset = sum(batch_tokens['ntokens'])

The Foundation Model Training API internally ingests the dataset and runs tokenization with the selected tokenizer. 
The output dataset is a collection of samples. Each sample is a collection of token ids represented as integers.  
We generate a histogram that visualizes the distribution of frequency of token counts in samples in the dataset. 
The visualization aids in identifying patterns, outliers, and central tendencies in the token distribution.

In [ ]:
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be used by the model during training")
print(f"Assume you'll train for {n_epochs} epochs on this dataset")
print(f"Then ~{n_epochs * n_billing_tokens_in_dataset} tokens will be running through the model during training")
plot_hist(pd.Series(batch_tokens['ntokens']))

## Continued pretraining

### Continued pretraining API arguments configuration

Similar to instruction finetuning, you need to specify the following:

**Foundation Model Training API arguments (FMT_API_args):**

- model: Specifies the model to be used for training. E.g., 'EleutherAI/gpt-neox-20b'
- train_data_path: The path to the training data. We currently only support a (remote/local) path to a collection of .txt files.
- task_type: Defines the type of task for which the training strategy will be applied. It is either 'INSTRUCTION_FINETUNE' or 'CONTINUED_PRETRAIN'.
- training_duration: The duration of the training process, expressed in numerical terms (e.g., 3) with units of training epochs.
- context_length: Specifies the context length of the model, set to 2048. This determines how many tokens the model considers for each training example. For Continued Pretraining, we concatenate tokens to form samples of length equal to context_length

**Temporary data path configuration:**

- temporary_mds_output_path: Defines a filesystem path where notebook running data can be stored. You need to make sure the path should not be shared by other users on the cluster, as it costs contention. For example, you can make it distinguishable by adding your username to the path.

**[Models supported by Foundation Model Training API](https://docs.mosaicml.com/projects/mcli/en/latest/finetuning/finetuning.html#supported-models):**. 

You need to specify context length based on the model mapping below.
```
ft_models = {
  'meta-llama/Llama-2-13b-hf': 4096,
  'meta-llama/Llama-2-7b-hf': 4096,
  'meta-llama/Llama-2-70b-hf': 4096,
  'codellama/CodeLlama-7b-hf': 16384,
  'codellama/CodeLlama-13b-hf': 16384,
  'codellama/CodeLlama-34b-hf': 16384,
  'mistralai/Mistral-7B-v0.1': 32768,
}
```

In [ ]:
FMT_API_args = Namespace(
    model= 'meta-llama/Llama-2-7b-chat-hf',
    train_data_path= os.path.join(home, 'ABT'), # this is the path to your collection of txt files
    task_type='CONTINUED_PRETRAIN',
    training_duration=3,
    context_length=8,
)
temporary_mds_output_path = os.path.join(home, 'mds_data_11Jan24_5')

Generate a synthetic dataset. Replace `train_data_path` with your raw data path.

In [ ]:
def generate_synthetic_dataset(folder_path, num_files=128):
    """Generate a synthetic dataset of text files with random words."""
    def generate_random_words(num_words=50):
        words = ["apple", "banana", "cherry", "date", "elderberry", "fig", "grape", "honeydew", "kiwi", "lemon", "mango", "nectarine", "orange", "papaya", "quince", "raspberry", "strawberry", "tangerine", "ugli", "vanilla", "watermelon", "xigua", "yam", "zucchini"]
        return ' '.join(random.choice(words) for _ in range(num_words))

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    for i in range(num_files):
        file_path = os.path.join(folder_path, f"file_{i}.txt")
        with open(file_path, 'w') as file:
            file.write(generate_random_words())

    print(f"Generated {num_files} files in '{folder_path}'.")

generate_synthetic_dataset(FMMT_API_args.train_data_path)

In [ ]:
!rm -rf {temporary_mds_output_path}

### Ingestion, tokenization, and materialization

Continued pretraining takes a folder of .txt files as input. It tokenizes the text fields and materializes as a streaming dataset of MDS format. 

The Foundation Model Training API uses [llmfoundry/scripts/data_prep/convert_text_to_mds.py](https://github.com/mosaicml/llm-foundry/blob/main/scripts/data_prep/convert_text_to_mds.py) to download all the txt files and convert them to MDS. 

In this notebook, we provide two additional approaches using Spark and Dask. 

**Important** Continued pretraining datasets are often much larger than instruction fine-tuning, so the tokenization and materialization can be very time consuming. 

In [ ]:
import os
os.makedirs(temporary_mds_output_path, exist_ok=True)

In [ ]:
cfg, tokenizer = create_om_cfg(FMT_API_args)

input_folder = FMT_API_args.train_data_path
output_folder = temporary_mds_output_path
concat_tokens = FMT_API_args.context_length
tokenizer_name = FMT_API_args.model

# Run convert_text_to_mds.py and dump MDS dataset to "save_folder"
args = parse_args(tokenizer_name, concat_tokens, output_folder, input_folder)

args.processes = 2
args.reprocess = True

n_samples = convert_text_to_mds(tokenizer_name=args.tokenizer,
                                                  output_folder=args.output_folder,
                                                  input_folder=args.input_folder,
                                                  concat_tokens=args.concat_tokens,
                                                  eos_text=args.eos_text,
                                                  bos_text=args.bos_text,
                                                  no_wrap=args.no_wrap,
                                                  compression=args.compression,
                                                  processes=args.processes,
                                                  reprocess=args.reprocess,
                                                  args_str=_args_str(args))

n_billing_tokens_in_dataset = n_samples * concat_tokens

### Token estimation

In [ ]:
MAX_TOKENS_PER_EXAMPLE = FMT_API_args.context_length if FMT_API_args.context_length is not None else 4096
TARGET_EPOCHS = FMT_API_args.training_duration if FMT_API_args.training_duration is not None else 1 
n_epochs = TARGET_EPOCHS

print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, ~{n_epochs * n_billing_tokens_in_dataset} tokens will be used in training")